In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Importing libraries
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn
import torchtext
import re

In [ ]:
# Function for dataset class
class bookDataset(Dataset):
    def __init__(self,images_file,labels_file,dataDir):
        self.inputs = pd.read_csv(os.path.join(dataDir,images_file))
        self.labels = pd.read_csv(os.path.join(dataDir,labels_file))
        self.img_dir = os.path.join(dataDir,"images","images")
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self,idx):
        """
        img_path = os.path.join(self.img_dir,self.inputs.iloc[idx,1])
        thisImg = Image.open(img_path)
        thisTransform = transforms.Compose([transforms.PILToTensor()])
        img_tensor = thisTransform(thisImg)/255
        """
        text = self.inputs.iloc[idx,2]
        word_list = re.split(' |,|\\.|\n|:|;|"|\'|`|{{|}}|[|]|\)|\(',text)
        
        
        thisLab = self.labels.iloc[idx,1]
        return img_tensor,thisLab

# dataset & dataloaders initialization
trainDataset = bookDataset("train_x.csv","train_y.csv","/kaggle/input/col774-2022/")
testDataset = bookDataset("non_comp_test_x.csv","non_comp_test_y.csv","/kaggle/input/col774-2022/")
trainloader = DataLoader(trainDataset,batch_size = 64,shuffle = True)
testloader = DataLoader(testDataset,batch_size = 64,shuffle = True)